<a href="https://colab.research.google.com/github/AlexxLai/ml0716/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()   # upload kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c titanic

In [ ]:
# 若上述無法使用，這使用此部分程式碼
# from urllib.request import urlretrieve
# url = "https://github.com/Elwing-Chou/ml0716/raw/master/train.csv"
# urlretrieve(url, "train.csv")

In [78]:
import pandas as pd
train_df = pd.read_csv("train.csv",encoding = "utf-8")
test_df = pd.read_csv("test.csv",encoding = "utf-8")
# train_df
# test_df

In [91]:
# concat: 把兩個dataframe統整在一起
# concat: 0[把行合併在一起↓] 1[把列合併在一起→]
datas = pd.concat([train_df,test_df])
datas = datas.drop(["PassengerId","Survived"],axis=1) # drop:捨去不必要的欄位,axis=1行捨去
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [92]:
# Step1. 補缺失值
# isna(): False: 有值, True: 有缺失值
s = datas.isna().sum() # Cabin缺失值太多(趨近資料集總數)，因此選擇補次多的Age、Fare...
# 篩選 seires[[True...]]，將值是True的以Series列出來
s[s > 0]
# Step2. 類型判斷(打開csv確認class)
# 數字(大小): Age,SibSp,Parch,Fare
# 類型(固定種類): Pclass,Name,Sex,Embarked


Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [93]:
# 預處理
# 1.補缺失值: 類型:補最常(可能)出現/不補(直接One-Hot: 補0)
# most = datas["Embarked"].value_counts() # 因為結果出來出來發現S最多，因此補S(最常/最有可能)出現
most = datas["Embarked"].value_counts().idxmax() # 與下段結果相同idxmax→index的max值
datas["Embarked"] = datas["Embarked"].fillna(most) # 將最(常出現/可能)的值補進去資料裡，因此是S
# 1.補缺失值: 數值:補最可能的值(找中位數)，因為找平均值的話會受到極端值的影響
med = datas.median().drop(["Pclass"]) # 在先前處理class時，經由確認後知道Pclass是類型而非數值
d = datas.fillna(med)
d.isna().sum()

Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin       1014
Embarked       0
dtype: int64

In [52]:
# eg.要抓取Mr. Miss等比較好做分類的關鍵字
# n = "Myles, Mr. Thomas Francis"
# split(",")[-1]是以分割符號為基準，保留符號後最後一段
# n = n.split(",")
# n = n.split(",")[-1]
# n = n.split(",")[-1].split(".")
# n = n.split(",")[-1].split(".")[0]
# n.strip() # 此語法可以將左右所有的空白全部剃除
# n

' Mr. Thomas Francis'

In [94]:
# 2. One-Hot(類別)
# 類別型態"絕對"不能轉為0,1,2...的形式(超過2種以上，類別沒有大小關係)，邏輯出現問題
#   S C Q
# S 1 0 0
# C 0 1 0
# Q 0 0 1
# 此為One-Hot 的形式

# 先處理name跟ticket
# name裡面,跟.(因為都有)，拿來做處理
def name(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip() # 此語法可以將左右所有的空白全部剃除
v = datas["Name"].apply(name).value_counts() # value_counts()此語法自動排序由值高至值低
white = v.index[:4] # 將數值太少的去除(重要度/顯著程度不高)

In [95]:
def name(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in white:
        return n
    else:
        return None
datas["Name"] = datas["Name"].apply(name)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,None,female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,Mr,male,NaN,0,0,359309,8.0500,NaN,S


In [96]:
v = datas["Ticket"].value_counts()
def ticket(t):
    global v
    return v[t]
datas["Ticket"] = datas["Ticket"].apply(ticket)

In [97]:
datas = datas.drop(["Cabin"],axis = 1)

In [98]:
# 如果想要重製index: datas.reset_index(drop=True)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C
2,3,Miss,female,26.0,0,0,1,7.9250,S
3,1,Mrs,female,35.0,1,0,2,53.1000,S
4,3,Mr,male,35.0,0,0,1,8.0500,S
...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,NaN,0,0,1,8.0500,S
414,1,None,female,39.0,0,0,3,108.9000,C
415,3,Mr,male,38.5,0,0,1,7.2500,S
416,3,Mr,male,NaN,0,0,1,8.0500,S


In [99]:
datas = pd.get_dummies(datas)
datas

,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,2,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,NaN,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
414,1,39.0,0,0,3,108.9000,0,0,0,0,1,0,1,0,0
415,3,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,1
416,3,NaN,0,0,1,8.0500,0,0,1,0,0,1,0,0,1


In [88]:
# iloc: 取列(依實際是第幾列), loc:(依列編號)
# [第一列,第二列...]
x_train = datas.iloc[:len(train_df)]
x_predict = datas.iloc[len(train_df):]
y_train = train_df["Survived"]